In [1]:
import os
from langchain_groq import ChatGroq
import google.generativeai as genai
from langchain.document_loaders import UnstructuredPDFLoader, PyPDFLoader
from langchain_pinecone import PineconeVectorStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import LLMChain
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from pinecone import Pinecone
from langchain.schema import Document
from langchain_core.embeddings import Embeddings
from typing import List
import numpy as np

from datasets import load_dataset
import pandas as pd
import os
import numpy as np
import time
from tqdm import tqdm

In [ ]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '<YOUR API KEY>'
os.environ['GROQ_API_KEY'] = '<YOUR API KEY>'
os.environ['PINECONE_API_KEY'] = '<YOUR API KEY>'
os.environ['PINECONE_ENV'] = 'gemini-rag'
os.environ["GEMINI_API_KEY"] = '<YOUR API KEY>'

In [3]:
from embeddings import GeminiEmbeddings
from db_utils import upsert , load_existing
from scrape_chain import process_query
from templates import make_templates
from qa_chains import QA_chain_with_websearch , create_base_chains

In [4]:
genai.configure(api_key=os.environ["GEMINI_API_KEY"])
embeddings = GeminiEmbeddings(api_key=os.environ["GEMINI_API_KEY"])
pc = Pinecone(os.environ['PINECONE_API_KEY'])

In [5]:
from langchain_pinecone import PineconeVectorStore
vectorstore = load_existing(embeddings)

In [6]:
answer_template , clarify_template , summary_template , web_summary_template, final_summary_template = make_templates()

In [7]:
answer_chain , clarify_chain , summary_chain , web_summary_chain , final_summary_chain = create_base_chains(answer_template , clarify_template , summary_template , web_summary_template, final_summary_template)

/Users/aditya/Developer/Aditya/Hybrid RAG/qa_chains.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  answer_chain = LLMChain(


In [8]:
question = 'I am having high fever and nausea since this morning.'
QA_chain_with_websearch(question , vectorstore, answer_chain , clarify_chain , summary_chain , web_summary_chain , final_summary_chain)

/Users/aditya/Developer/Aditya/Hybrid RAG/qa_chains.py:47: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  a = answer_chain.run(question=q, context=ctx)


Searching The web!


"Here is a concise and detailed final summary of the information:\n\n**Fever Management Guide**\n\nIf you have a high fever, it is recommended to take Paracetamol for fever relief and stay hydrated. If your symptoms worsen, you should visit the Emergency Room for further evaluation. (Corpus Text)\n\nAdditionally, according to a web search, a fever of 104°F and above is considered high. For high fevers, take acetaminophen first, and if it doesn't go down in about 30 minutes, reach out to your doctor or go to urgent care. (Dr. Ruff, Web Source)\n\nPlease note that the additional information from the web search is provided for further guidance, but the primary advice is based on the corpus text."